# GMS를 통한 Imagen 이미지 생성 API

GMS (GenAI Management Service)를 사용하여 Google Imagen API로 이미지를 생성합니다.

## GMS API 구조

```
GMS 엔드포인트: https://gms.ssafy.io/gmsapi/
+ 기존 API 경로: generativelanguage.googleapis.com/v1beta/models/...
+ Query String: ?key=$GMS_KEY
```


## 방법 1: REST API 직접 호출 (추천)


In [1]:
%pip install google-genai

  Obtaining dependency information for google-genai from https://files.pythonhosted.org/packages/ec/66/03f663e7bca7abe9ccfebe6cb3fe7da9a118fd723a5abb278d6117e7990e/google_genai-1.52.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/46.8 kB ? eta -:--:--
     ---------------- --------------------- 20.5/46.8 kB 330.3 kB/s eta 0:00:01
     ------------------------ ------------- 30.7/46.8 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 46.8/46.8 kB 468.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   ----------------------- ---------------- 153.6/261.2 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 261.2/261.2 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
import json
import base64
import os
from datetime import datetime

def generate_gemini_image(api_key, prompt):
    # 1. GMS 프록시 엔드포인트 URL 설정 (모델명: gemini-2.0-flash-exp-image-generation)
    url = f"https://gms.ssafy.io/gmsapi/generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp-image-generation:generateContent?key={api_key}"

    # 2. 헤더 설정
    headers = {
        "Content-Type": "application/json"
    }

    # 3. 요청 데이터(Payload) 구성
    # 이 모델은 'responseModalities'를 설정하여 텍스트와 이미지를 동시에 요청할 수 있습니다.
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ],
        "generationConfig": {
            "responseModalities": ["TEXT", "IMAGE"]
        }
    }

    try:
        # 4. POST 요청 보내기
        print(f"생성 요청 중... (Prompt: {prompt})")
        response = requests.post(url, headers=headers, data=json.dumps(payload))

        # 5. 응답 확인
        if response.status_code == 200:
            response_data = response.json()
            
            # 응답 데이터 구조 파싱
            candidates = response_data.get('candidates', [])
            if not candidates:
                print("생성 실패: 후보(candidates) 결과가 없습니다.")
                return

            # 첫 번째 후보의 내용 가져오기
            parts = candidates[0].get('content', {}).get('parts', [])
            
            save_dir = "generated_results"
            os.makedirs(save_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            image_count = 0
            
            # parts 리스트를 순회하며 텍스트와 이미지를 분리 처리
            for part in parts:
                # 텍스트가 있는 경우 출력
                if 'text' in part:
                    print(f"\n[모델 응답 텍스트]:\n{part['text']}\n")
                
                # 이미지가 있는 경우 저장 (inlineData)
                if 'inlineData' in part:
                    mime_type = part['inlineData'].get('mimeType', 'image/png')
                    image_data_b64 = part['inlineData']['data']
                    
                    # 확장자 결정
                    ext = mime_type.split('/')[-1]
                    if ext == 'jpeg': ext = 'jpg'
                    
                    image_count += 1
                    filename = f"{save_dir}/gemini_image_{timestamp}_{image_count}.{ext}"
                    
                    # Base64 디코딩 및 저장
                    with open(filename, "wb") as f:
                        f.write(base64.b64decode(image_data_b64))
                    print(f"[이미지 저장 완료]: {filename}")

            if image_count == 0:
                print("생성된 이미지가 없습니다. (텍스트만 반환되었을 수 있음)")

        else:
            print(f"에러 발생: Status Code {response.status_code}")
            print(response.text)

    except Exception as e:
        print(f"요청 중 예외 발생: {e}")

# --- 실행부 ---
if __name__ == "__main__":
    # 여기에 발급받은 GMS API 키를 입력하세요
    GMS_KEY = "S14P02AQ06-88cb9f96-1505-4bb8-b068-57fe14afebee"
    
    # 생성할 이미지 프롬프트
    DEFAULT_PROMPT = "해당 꿈 내용을 읽고 꿈을 영화의 한 장면처럼 이미지를 생성해줘 이미지는 정사각형으로 만들어줘. 꿈 내용 : "
    USER_PROMPT = f"{DEFAULT_PROMPT} 똥통에 빠지는 꿈을 꿨어"
    
    generate_gemini_image(GMS_KEY, USER_PROMPT)

생성 요청 중... (Prompt: 해당 꿈 내용을 읽고 꿈을 영화의 한 장면처럼 이미지를 생성해줘 이미지는 정사각형으로 만들어줘. 꿈 내용 :  똥통에 빠지는 꿈을 꿨어)

[모델 응답 텍스트]:
A wide-angle shot of a person falling headfirst into an overflowing, old-fashioned outhouse toilet filled with dark, murky excrement. The person's arms and legs are flailing as they disappear into the sludge, with only bubbles rising to the surface. The wooden planks of the outhouse are weathered and stained. The scene is dimly lit, casting long shadows.



[이미지 저장 완료]: generated_results/gemini_image_20251128_163514_1.png
